In [1]:
import pandas as pd
import numpy as np

import pandas as pd
import numpy as np

class display(object):
    """Display HTML representation of multiple objects"""

    template = """<div style="float: left; padding: 10px;">
    <p style='font-family:"Courier New", Courier, monospace'>{0}</p>{1}
    </div>"""
    
    def __init__(self, *args):
        self.args = args
    def _repr_html_(self):
        return '\n'.join(self.template.format(a, eval(a)._repr_html_())
            for a in self.args)
    def __repr__(self):
        return '\n\n'.join(a + '\n' + repr(eval(a))
            for a in self.args)

In [2]:
pop = pd.read_csv('state-population.csv')
areas = pd.read_csv('state-areas.csv')
abbrevs = pd.read_csv('state-abbrevs.csv')
display('pop.head()', 'areas.head()', 'abbrevs.head()')

pop.head()
  state/region     ages  year  population
0           AL  under18  2012     1117489
1           AL    total  2012     4817528
2           AL  under18  2010     1130966
3           AL    total  2010     4785570
4           AL  under18  2011     1125763

areas.head()
        state  area (sq. mi)
0     Alabama          52423
1      Alaska         656425
2     Arizona         114006
3    Arkansas          53182
4  California         163707

abbrevs.head()
        state abbreviation
0     Alabama           AL
1      Alaska           AK
2     Arizona           AZ
3    Arkansas           AR
4  California           CA

In [3]:
merged = pd.merge(pop, abbrevs, left_on='state/region', right_on='abbreviation', how='outer')
merged = merged.drop('abbreviation', axis=1) # drop duplicate info
merged.head()

,state/region,ages,year,population,state
0,AL,under18,2012,1117489,Alabama
1,AL,total,2012,4817528,Alabama
2,AL,under18,2010,1130966,Alabama
3,AL,total,2010,4785570,Alabama
4,AL,under18,2011,1125763,Alabama


In [4]:
merged[merged['population'].isnull()].head()

,state/region,ages,year,population,state
2448,PR,under18,1990,NaN,NaN
2449,PR,total,1990,NaN,NaN
2450,PR,total,1991,NaN,NaN
2451,PR,under18,1991,NaN,NaN
2452,PR,total,1993,NaN,NaN


In [5]:
merged['state/region'][merged['state'].isnull()].unique()

array(['PR', 'USA'], dtype=object)

In [6]:
merged.loc[merged['state/region'] == 'PR', 'state'] = 'Puerto Rico'
merged.loc[merged['state/region'] == 'USA', 'state'] = 'United States'
merged.isnull().any()

state/region    False
ages            False
year            False
population       True
state           False
dtype: bool

In [8]:
final = pd.merge(merged, areas, on='state', how='left')
final.head()


,state/region,ages,year,population,state,area (sq. mi)
0,AL,under18,2012,1117489,Alabama,52423
1,AL,total,2012,4817528,Alabama,52423
2,AL,under18,2010,1130966,Alabama,52423
3,AL,total,2010,4785570,Alabama,52423
4,AL,under18,2011,1125763,Alabama,52423


In [10]:
final['state'][final['area (sq. mi)'].isnull()].unique()

array(['United States'], dtype=object)

In [11]:
final.dropna(inplace=True)
final.head()

,state/region,ages,year,population,state,area (sq. mi)
0,AL,under18,2012,1117489,Alabama,52423
1,AL,total,2012,4817528,Alabama,52423
2,AL,under18,2010,1130966,Alabama,52423
3,AL,total,2010,4785570,Alabama,52423
4,AL,under18,2011,1125763,Alabama,52423


In [27]:
data2010 = final.query("year == 2010 & ages == 'total'")
data2010.head()

,state/region,ages,year,population,state,area (sq. mi)
3,AL,total,2010,4785570,Alabama,52423
91,AK,total,2010,713868,Alaska,656425
101,AZ,total,2010,6408790,Arizona,114006
189,AR,total,2010,2922280,Arkansas,53182
197,CA,total,2010,37333601,California,163707


In [28]:
data2010.set_index('state', inplace=True)
density = data2010['population'] / data2010['area (sq. mi)']
density.sort_values(ascending=False)
density.head()

state
Alabama        91.287603
Alaska          1.087509
Arizona        56.214497
Arkansas       54.948667
California    228.051342
dtype: float64

In [29]:
density.tail()

state
Washington         94.557817
West Virginia      76.519582
Wisconsin          86.851900
Wyoming             5.768079
Puerto Rico      1058.665149
dtype: float64

In [32]:
data2010.loc('

,state/region,ages,year,population,area (sq. mi)
state,,,,,
Alabama,AL,total,2010,4785570,52423
Alaska,AK,total,2010,713868,656425
Arizona,AZ,total,2010,6408790,114006
Arkansas,AR,total,2010,2922280,53182
California,CA,total,2010,37333601,163707
